In [ ]:
from collections import Counter
from itertools import chain
import mmh3
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from htools import hdir

In [ ]:
chain.from_iterable(row.split(' ') for row in sents)

In [ ]:
sents = [
    'I walked to the store so I hope it is not closed.',
    'The theater is closed today and the sky is grey.',
    'His dog is brown while hers is grey.'
]
labels = [0, 1, 1]

For now, just convert int to str and take hash. Another option that is meant for ints is Knuth's multiplicative method:

hash(i) = i*2654435761 mod 2^32

In [ ]:
mmh3.hash('0', signed=False)

3530670207

In [ ]:
[i if i % 2 == 0 else i*10 for i in range(10)]

[0, 10, 2, 30, 4, 50, 6, 70, 8, 90]

In [ ]:
def hash_int(x, n_buckets, n_hashes=3):
    """Slightly hacky way to probabilistically hash an integer by
    first converting it to a string.
    
    Parameters
    ----------
    x: int
        The integer to hash.
    n_buckets: int
        The number of buckets that items will be mapped to. Typically 
        this would occur outside the hashing function, but since 
        the intended use case is so narrow here it makes sense to me 
        to include it here.
    n_hashes: int
        The number of times to hash x, each time with a different seed.
        
        
    Returns
    -------
    list[int]: A list of integers with length `n_hashes`, where each integer
        is in [0, n_buckets).
    """
    assert isinstance(x, int), 'Input `x` must have type int.'
    return [mmh3.hash(str(x), i, signed=False) % n_buckets for i in range(n_hashes)]

In [ ]:
def hash_int_tensor(x_r2, n_buckets, n_hashes=3, pad_idx=0):
    """Hash a rank 2 LongTensor.
    
    Parameters
    ----------
    x_r2: torch.LongTensor
        Rank 2 tensor of integers. Shape: (bs, seq_len)
    n_buckets: int
        Number of buckets to hash items into (i.e. the number of 
        rows in the embedding matrix). Typically a moderately large
        prime number, like 251 or 997.
    n_hashes: int
        Number of hashes to take for each input index. This determines
        the number of rows of the embedding matrix that will be summed
        to get the representation for each word. Typically 2-5.
    pad_idx: int or None
        If you want to pad sequences with vectors of zeros, pass in an
        integer (same as the `padding_idx` argument to nn.Embedding).
        If None, no padding index will be used. The sequences must be
        padded before passing them into this function.
        
    Returns
    -------
    torch.LongTensor: Tensor of indices where each row corresponds
        to one of the input indices. Shape: (bs, seq_len, n_hashes)
    """
    return torch.tensor(
        [[hash_int(x.item(), n_buckets, n_hashes) 
          if x != pad_idx else [pad_idx]*n_hashes for x in row]
         for row in x_r2]
    )

In [ ]:
for row in x:
    print(row, end='\n\n')
    print([x.item() for x in row])

tensor([ 2,  5,  6,  3,  7,  8,  2,  9, 10,  1])

[2, 5, 6, 3, 7, 8, 2, 9, 10, 1]
tensor([13, 14,  1, 15, 16, 17,  3, 18,  1,  4])

[13, 14, 1, 15, 16, 17, 3, 18, 1, 4]
tensor([19, 20,  1, 21, 22, 23,  1,  4,  0,  0])

[19, 20, 1, 21, 22, 23, 1, 4, 0, 0]


In [ ]:
hash_int_tensor(x, 11)

torch.Size([3, 10, 3])

In [ ]:
hash_int_tensor(x[0, None], 11)

tensor([[[ 8,  2,  7],
         [ 2,  8,  1],
         [ 6,  6, 10],
         [10,  5,  5],
         [ 6,  9,  7],
         [ 5,  9,  4],
         [ 8,  2,  7],
         [ 5, 10,  8],
         [ 7,  8,  6],
         [ 6, 10,  6]]])

In [ ]:
hash_int_tensor(x[2,  None], 11)

tensor([[[ 2, 10,  6],
         [ 1,  7,  8],
         [ 6, 10,  6],
         [ 2,  7,  1],
         [ 9,  4,  8],
         [ 5,  3,  3],
         [ 6, 10,  6],
         [ 4,  8,  6],
         [ 0,  0,  0],
         [ 0,  0,  0]]])

In [ ]:
x

tensor([[ 2,  5,  6,  3,  7,  8,  2,  9, 10,  1],
        [13, 14,  1, 15, 16, 17,  3, 18,  1,  4],
        [19, 20,  1, 21, 22, 23,  1,  4,  0,  0]])

In [ ]:
[hash_int(n.item(), 11) for n in x[0]]

[[8, 2, 7],
 [2, 8, 1],
 [6, 6, 10],
 [10, 5, 5],
 [6, 9, 7],
 [5, 9, 4],
 [8, 2, 7],
 [5, 10, 8],
 [7, 8, 6],
 [6, 10, 6]]

In [ ]:
for i in range(0, 200, 17):
    print(hash_int(i, 11))

[9, 8, 1]
[2, 6, 9]
[8, 2, 7]
[2, 5, 7]
[10, 10, 2]
[0, 10, 4]
[3, 8, 4]
[6, 5, 0]
[6, 10, 8]
[4, 4, 0]
[7, 1, 0]
[10, 4, 6]


In [ ]:
class Data(Dataset):
    
    def __init__(self, sentences, labels, seq_len):
        x = [s.split(' ') for s in sentences]
        self.w2i = self.make_w2i(x)
        self.seq_len = seq_len
        self.x = self.encode(x)
        self.y = torch.tensor(labels)
        
    def __getitem__(self, i):
        return self.x[i], self.y[i]
    
    def __len__(self):
        return len(self.y)
    
    def make_w2i(self, tok_rows):
        return {k: i for i, (k, v) in 
                enumerate(Counter(chain(*tok_rows)).most_common(), 1)}
    
    def encode(self, tok_rows):
        enc = np.zeros((len(tok_rows), self.seq_len), dtype=int)
        for i, row in enumerate(tok_rows):
            trunc = [self.w2i.get(w, 0) for w in row[:self.seq_len]]
            enc[i, :len(trunc)] = trunc
        return torch.tensor(enc)

In [ ]:
ds = Data(sents, labels, 10)
ds[1]

(tensor([13, 14,  1, 15, 16, 17,  3, 18,  1,  4]), tensor(1))

In [ ]:
dl = DataLoader(ds, batch_size=3)
x, y = next(iter(dl))
x, y

(tensor([[ 2,  5,  6,  3,  7,  8,  2,  9, 10,  1],
         [13, 14,  1, 15, 16, 17,  3, 18,  1,  4],
         [19, 20,  1, 21, 22, 23,  1,  4,  0,  0]]), tensor([0, 1, 1]))

In [ ]:
x.shape

torch.Size([3, 10])

In [ ]:
ds.x

tensor([[ 2,  5,  6,  3,  7,  8,  2,  9, 10,  1],
        [13, 14,  1, 15, 16, 17,  3, 18,  1,  4],
        [19, 20,  1, 21, 22, 23,  1,  4,  0,  0]])

In [ ]:
ds.w2i

{'is': 1,
 'I': 2,
 'the': 3,
 'grey.': 4,
 'walked': 5,
 'to': 6,
 'store': 7,
 'so': 8,
 'hope': 9,
 'it': 10,
 'not': 11,
 'closed.': 12,
 'The': 13,
 'theater': 14,
 'closed': 15,
 'today': 16,
 'and': 17,
 'sky': 18,
 'His': 19,
 'dog': 20,
 'brown': 21,
 'while': 22,
 'hers': 23}

# To do:
- [x] maybe handle padding differently (i.e. just a row of zeros as usual. Should this happen in hash_int, hash_int_tensor, or BloomEmbedding?)
- maybe use fastai embedding() instead of nn.Embedding? Check if the weight init method might help for this.
- maybe use nn.embeddingbag?
- experiment with different numbers of embeddings and hashes. Try to find guidelines for what reasonable choices are to prevent collisions. 
    - Eventually, maybe better to let user input vocab size and choose prob of collision, then automatically select values for n_emb and n_hashes. 
- check to make sure indices are working correctly after switching hash_int_tensor to take only 2d tensors. Also consider if this is the preferred way to do this.
- should we let user choose between mean and sum? Wonder if mean would be better bc we could try different values of n_hashes while still loading pre-trained embeddings (bc scale is standardized)? But that probably doesn't work bc the hashes will be different anyway.

In [ ]:
class BloomEmbedding(nn.Module):
    """Bloom Embedding layer for memory-efficient word representations.
    Each word is encoded by a combination of rows of the embedding
    matrix. The number of rows can therefore be far lower than the number
    of words in our vocabulary while still providing unique representations.
    
    The reduction in rows allows us to use memory in other ways: 
    a larger embedding dimension, more or larger layers after the embedding,
    or larger batch sizes.
    """
    
    def __init__(self, n_emb=251, emb_dim=100, n_hashes=4, padding_idx=0):
        """
        Parameters
        ----------
        n_emb: int
            Number of rows to create in the embedding matrix. A prime
            number is recommended. Lower numbers will be more 
            memory-efficient but increase the chances of collisions.
        emb_dim: int
            Size of each embedding. If emb_dim=100, each word will
            be represented by a 100-dimensional vector.
        n_hashes: int
            This determines the number of hashes that will be taken
            for each word index, and as a result, the number of rows
            that will be summed to create each unique representation.
            The higher the number, the lower the chances of a collision.
        padding_idx: int or None
            If an integer is provided, this will set aside the corresponding
            row in the embedding matrix as a vector of zeros. If None, no
            padding vector will be allocated.
            
        Suggested values for a vocab size of ~30,000:
        
        | n_emb | n_hashes | unique combos |
        |-------|----------|---------------|
        | 127   | 5        | 29,998        |
        | 251   | 4        | 29,996        |
        | 997   | 3        | 29,997        |
        | 5,003 | 2        | 29,969        |
        """
        super().__init__()
        self.n_emb = n_emb
        self.emb = nn.Embedding(n_emb, emb_dim, padding_idx=padding_idx)
        self.n_hashes = n_hashes
        self.pad_idx = padding_idx
        
    def forward(self, x):
        """
        Parameters
        ----------
        x: torch.LongTensor
            Input tensor of word indices. (bs x seq_len)
            
        Returns
        -------
        torch.FloatTensor: Words encoded with combination of embeddings.
            (bs x seq_len x emb_dim)
        """
        # (bs, seq_len, n_hashes)
        hashed = hash_int_tensor(x, self.n_emb, self.n_hashes, self.pad_idx)
        # (bs, seq_len, n_hashes, emb_dim) -> sum -> (bs, seq_len, emb_dim)
        return self.emb(hashed).sum(-2)

In [ ]:
be = BloomEmbedding(11, 4)
be.emb.weight

Parameter containing:
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-2.0462e+00,  5.2716e-01, -1.8577e-01,  2.0636e-01],
        [-2.7725e+00, -4.0539e-02, -1.2162e-01, -1.5453e+00],
        [ 1.4179e-01, -1.5752e+00,  1.1838e+00,  6.2302e-01],
        [ 6.3845e-01,  9.6330e-01,  1.0995e+00, -6.4528e-01],
        [-8.0821e-01,  1.3410e+00,  2.0342e-03, -1.4437e+00],
        [ 5.3196e-01,  4.9610e-01,  2.4020e-01,  9.6942e-01],
        [ 6.7312e-01, -1.6300e+00, -1.0742e+00, -4.6747e-01],
        [ 7.8946e-02, -1.1476e+00, -9.0362e-02,  3.1300e-01],
        [ 1.0806e+00, -9.5686e-01,  3.3655e-01, -2.7509e-01],
        [ 5.7283e-01,  7.7029e-01,  8.0080e-01,  2.0770e+00]],
       requires_grad=True)

In [ ]:
x

tensor([[ 2,  5,  6,  3,  7,  8,  2,  9, 10,  1],
        [13, 14,  1, 15, 16, 17,  3, 18,  1,  4],
        [19, 20,  1, 21, 22, 23,  1,  4,  0,  0]])

In [ ]:
for i in range(24):
    print(hash_int(i, 11))

[9, 8, 1]
[6, 10, 6]
[8, 2, 7]
[10, 5, 5]
[4, 8, 6]
[2, 8, 1]
[6, 6, 10]
[6, 9, 7]
[5, 9, 4]
[5, 10, 8]
[7, 8, 6]
[7, 9, 2]
[1, 7, 3]
[2, 5, 1]
[9, 8, 8]
[8, 1, 10]
[6, 3, 9]
[2, 6, 9]
[10, 5, 0]
[2, 10, 6]
[1, 7, 8]
[2, 7, 1]
[9, 4, 8]
[5, 3, 3]


In [ ]:
y = be(x)
y.shape

torch.Size([3, 10, 4])

In [ ]:
y[0]

tensor([[-1.9414e+00, -3.9658e+00, -1.3765e+00, -1.3868e+00],
        [-7.5122e+00, -7.0155e-01, -5.1937e-01, -2.5713e+00],
        [ 2.2096e+00,  2.5328e+00,  2.0820e+00,  6.0929e+00],
        [-1.8518e+00,  4.7934e+00,  8.0690e-01, -2.2542e+00],
        [-4.8675e-01, -2.1313e+00, -6.1906e-01, -1.3185e+00],
        [ 9.1088e-01,  1.3475e+00,  1.4381e+00, -2.3641e+00],
        [-1.9414e+00, -3.9658e+00, -1.3765e+00, -1.3868e+00],
        [ 9.2421e-01,  6.8116e-03,  1.0490e+00,  6.7120e-01],
        [-1.4884e+00, -2.3221e+00, -1.0460e+00, -7.3039e-01],
        [ 1.6367e+00,  1.7625e+00,  1.2812e+00,  4.0159e+00]],
       grad_fn=<SelectBackward>)

In [ ]:
y[1]

tensor([[-8.3994,  1.7871, -0.4270, -4.3281],
        [ 1.7705, -2.7560,  0.3960,  1.3203],
        [ 1.6367,  1.7625,  1.2812,  4.0159],
        [-1.2527, -1.4254,  1.7085,  3.2194],
        [ 1.7544, -2.0360,  1.7605,  1.3173],
        [-1.1599, -0.5013,  0.4551, -0.8510],
        [-1.8518,  4.7934,  0.8069, -2.2542],
        [-3.0078,  2.0708,  0.6812, -0.9121],
        [ 1.6367,  1.7625,  1.2812,  4.0159],
        [-1.5231,  0.2712,  1.1277, -0.9082]], grad_fn=<SelectBackward>)

In [ ]:
y[2]

tensor([[-1.0948,  1.9961,  1.7202,  3.5781],
        [-0.7622, -1.7544, -1.1101,  1.0213],
        [ 1.6367,  1.7625,  1.2812,  4.0159],
        [-3.5071, -0.1801, -0.2821, -2.4517],
        [ 2.3300, -0.6451,  1.5859,  0.3620],
        [-0.4457, -2.9571,  2.2792,  0.1153],
        [ 1.6367,  1.7625,  1.2812,  4.0159],
        [-1.5231,  0.2712,  1.1277, -0.9082],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000]], grad_fn=<SelectBackward>)

In [ ]:
for w, i in ds.w2i.items():
    print(w, i, be(torch.tensor([[i]])).detach().numpy().squeeze())
#           .emb.weight[hash_int(i, be.n_emb)])

is 1 [1.6367457 1.7624942 1.2811998 4.0158687]
I 2 [-1.9414458 -3.9658241 -1.3765308 -1.3868113]
the 3 [-1.851784   4.793353   0.8069003 -2.254175 ]
grey. 4 [-1.5230992  0.2712285  1.1277255 -0.9082023]
walked 5 [-7.5122128  -0.70155233 -0.5193659  -2.5713277 ]
to 6 [2.2095788 2.5327837 2.0819974 6.0928936]
store 7 [-0.48674607 -2.131305   -0.6190572  -1.3184783 ]
so 8 [ 0.91088253  1.3474642   1.438086   -2.364105  ]
hope 9 [0.9242083  0.00681156 1.0490168  0.67120016]
it 10 [-1.4884353  -2.3220832  -1.0459671  -0.73038846]
not 11 [-0.34558553 -4.257416   -1.9334465  -2.7553678 ]
closed. 12 [-0.6585116 -1.9077785  0.7246287  2.4389334]
The 13 [-8.399364    1.7871077  -0.42696917 -4.32806   ]
theater 14 [ 1.7704833  -2.7560353   0.39602363  1.3203299 ]
closed 15 [-1.2526826 -1.4254087  1.7084544  3.2194   ]
today 16 [ 1.7543795 -2.0359812  1.7605362  1.3173498]
and 17 [-1.159863  -0.5012967  0.455131  -0.8510109]
sky 18 [-3.0078273   2.0707722   0.6812143  -0.91205084]
His 19 [-1.09483

In [ ]:
hashed = hash_int_tensor(torch.tensor([23]).unsqueeze(0), 11, 4)
hashed

tensor([[[5, 3, 3, 8]]])

In [ ]:
be.emb.weight[hashed].sum(2)

tensor([[[-0.4457, -2.9571,  2.2792,  0.1153]]], grad_fn=<SumBackward1>)

In [ ]:
def unique_combos(tups):
    return len(set(tuple(sorted(x)) for x in tups))

In [ ]:
def hash_all_idx(vocab_size, n_buckets, n_hashes):
    return [hash_int(i, n_buckets, n_hashes) for i in range(vocab_size)]

In [ ]:
buckets2hashes = {127: 5,
                  251: 4,
                  997: 3,
                  5_003: 2}
for b, h in buckets2hashes.items():
    tups = hash_all_idx(30_000, b,  h)
    unique = unique_combos(tups)
    print('Buckets:', b, 'Hashes:', h, 'Unique combos:', unique,
          '% unique:', round(unique/30_000, 5))

Buckets: 127 Hashes: 5 Unique combos: 29998 % unique: 0.99993
Buckets: 251 Hashes: 4 Unique combos: 29996 % unique: 0.99987
Buckets: 997 Hashes: 3 Unique combos: 29997 % unique: 0.9999
Buckets: 5003 Hashes: 2 Unique combos: 29969 % unique: 0.99897


In [ ]:
def eval_n_buckets(vocab_size, hash_sizes, bucket_sizes):
    for bs in bucket_sizes:
        for hs in hash_sizes:
            tups = hash_all_idx(vocab_size, bs, hs)
            unique = unique_combos(tups)
            print(bs, hs, round(unique/vocab_size, 4))

In [ ]:
eval_n_buckets(80, range(2, 6), [5, 11, 19, 29, 37])

5 2 0.1875
5 3 0.3625
5 4 0.5375
5 5 0.625
11 2 0.625
11 3 0.875
11 4 0.975
11 5 1.0
19 2 0.85
19 3 0.975
19 4 1.0
19 5 1.0
29 2 0.9375
29 3 0.9875
29 4 1.0
29 5 1.0
37 2 0.925
37 3 1.0
37 4 1.0
37 5 1.0


In [ ]:
x = torch.randint(0, 30_000, (64, 100))
x.shape

torch.Size([64, 100])

In [ ]:
%%timeit -n 5 -r 5
hashed = hash_int_tensor(x, 127, 5)

71.6 ms ± 3.83 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [ ]:
%%timeit -n 5 -r 5
hashed = hash_int_tensor(x, 251, 4)

65.6 ms ± 2.54 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [ ]:
%%timeit -n 5 -r 5
hashed = hash_int_tensor(x, 997, 3)

60.9 ms ± 3.67 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [ ]:
%%timeit -n 5 -r 5
hashed = hash_int_tensor(x, 5_003, 2)

56.6 ms ± 3.26 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)
